<a href="https://colab.research.google.com/github/3zero69/-/blob/main/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%99%84%EB%B2%BD_%EA%B0%80%EC%9D%B4%EB%93%9C_ch8_7%2C_10_pg_516~528%2C_544~561.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

07. 문서 군집화 소개와 실습

* 문서 군집화
: 비슷한 텍스트 구성의 문서를 군집화 하는 것
  - 동일한 군집에 속하는 문서를 같은 카테고리 소속으로 분류할 수 있다는 점에서 텍스트 분류 기반의 문서 분류와 비슷
  - 그러나 비지도 학습 기반으로 동작작 

Opinion Review 데이터 세트를 이용한 문서 군집화 수행

In [8]:
from google.colab import drive
drive.mount("/content/drive")

# 경로 이동
%cd /content/drive/MyDrive/Colab Notebooks/ESAA_OB/topics

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/ESAA_OB/topics


In [9]:
import pandas as pd
import glob, os

#path로 지정한 디렉터리 밑에 있는 모든 .data 파일의 파일명을 리스트로 취합
all_files=glob.glob(os.path.join('*.data'))
filename_list=[]
opinion_text=[]

#개별 파일의 파일명은 filename_list로 취합,
#개별 파일의 파일 내용은 df 로딩 후 다시 string으로 변환해 opinion_text list로 취합합
for file_ in all_files:
  #개별 파일을 읽어서 df로 생성성
  df=pd.read_table(file_, index_col=None, header=0, encoding='latin1')

  #절대 경로로 주어진 파일명을 가공.
  #맨 마지막 .data 확장자도 제거거
  filename_=file_.split('/')[-1]
  filename=filename_.split('.')[0]

  #파일명 list와 파일 내용 list에 파일명과 파일 내용을 추가가
  filename_list.append(filename)
  opinion_text.append(df.to_string())

#파일명 list와 파일 내용 list 객체를 df로 생성성
document_df=pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()


,filename,opinion_text
0,buttons_amazon_kindle,...
1,display_garmin_nuvi_255W_gps,...
2,accuracy_garmin_nuvi_255W_gps,...
3,bathroom_bestwestern_hotel_sfo,...
4,battery-life_netbook_1005ha,...


In [10]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

# 단어 원형 추출 함수
lemmar = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# 특수 문자 사전 생성: {33: None ...}
# ord(): 아스키 코드 생성
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

# 특수 문자 제거 및 단어 원형 추출
def LemNormalize(text):
    # 텍스트 소문자 변경 후 특수 문자 제거
    text_new = text.lower().translate(remove_punct_dict)
    
    # 단어 토큰화
    word_tokens = nltk.word_tokenize(text_new)
    
    # 단어 원형 추출
    return LemTokens(word_tokens)

In [11]:
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect=TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', ngram_range=(1,2), min_df=0.05, max_df=0.85)
#opinion_text 칼럼 값으로 피처 벡터화 수행
feature_vect=tfidf_vect.fit_transform(document_df['opinion_text'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


In [12]:
from sklearn.cluster import KMeans

#5개 집합으로 군집화 수행
km_cluster=KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label=km_cluster.labels_
cluster_centers=km_cluster.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [13]:
document_df['cluster_label']=cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,buttons_amazon_kindle,...,1
1,display_garmin_nuvi_255W_gps,...,3
2,accuracy_garmin_nuvi_255W_gps,...,3
3,bathroom_bestwestern_hotel_sfo,...,0
4,battery-life_netbook_1005ha,...,1


In [14]:
#cluster_label=0인 데이터 - 호텔
document_df[document_df['cluster_label']==0].sort_values(by='filename')
     

,filename,opinion_text,cluster_label
3,bathroom_bestwestern_hotel_sfo,...,0
10,food_holiday_inn_london,...,0
15,food_swissotel_chicago,...,0
20,free_bestwestern_hotel_sfo,...,0
19,location_bestwestern_hotel_sfo,...,0
14,location_holiday_inn_london,...,0
17,parking_bestwestern_hotel_sfo,...,0
33,price_holiday_inn_london,...,0
36,room_holiday_inn_london,...,0
39,rooms_bestwestern_hotel_sfo,...,0


In [15]:
#cluster_label=1인 데이터 - 킨들, 아이팟, 넷북 등 포터블 전자기기
document_df[document_df['cluster_label']==1].sort_values(by='filename')
     

,filename,opinion_text,cluster_label
5,battery-life_amazon_kindle,...,1
9,battery-life_ipod_nano_8gb,...,1
4,battery-life_netbook_1005ha,...,1
0,buttons_amazon_kindle,...,1
31,performance_netbook_1005ha,...,1
38,price_amazon_kindle,...,1
46,speed_windows7,...,1


In [16]:
#cluster_label=2인 데이터 - 1과 비슷하지만 차량용 네이게이션
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
8,comfort_honda_accord_2008,...,2
7,comfort_toyota_camry_2007,...,2
25,gas_mileage_toyota_camry_2007,...,2
23,interior_honda_accord_2008,...,2
16,interior_toyota_camry_2007,...,2
12,mileage_honda_accord_2008,...,2
13,performance_honda_accord_2008,...,2
28,quality_toyota_camry_2007,...,2
26,seats_honda_accord_2008,...,2
48,transmission_toyota_camry_2007,...,2


In [17]:
#cluster_label=4인 데이터 - 전자기기 등등
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
22,eyesight-issues_amazon_kindle,...,4
18,features_windows7,...,4
24,fonts_amazon_kindle,...,4
21,keyboard_netbook_1005ha,...,4
11,navigation_amazon_kindle,...,4
40,screen_garmin_nuvi_255W_gps,...,4
30,screen_ipod_nano_8gb,...,4
41,screen_netbook_1005ha,...,4
32,size_asus_netbook_1005ha,...,4
45,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,4


전반적인 군집 개수가 많아 세분화 됨

중심 개수를 3개로 낮춰서 다시 보기

In [18]:

from sklearn.cluster import KMeans

#3개의 집합으로 군집화
km_cluster=KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label=km_cluster.labels_
cluster_centers=km_cluster.cluster_centers_

#소속 군집을 cluster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label']=cluster_label
document_df.sort_values(by='cluster_label')

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,filename,opinion_text,cluster_label
25,gas_mileage_toyota_camry_2007,...,0
28,quality_toyota_camry_2007,...,0
26,seats_honda_accord_2008,...,0
23,interior_honda_accord_2008,...,0
13,performance_honda_accord_2008,...,0
12,mileage_honda_accord_2008,...,0
8,comfort_honda_accord_2008,...,0
16,interior_toyota_camry_2007,...,0
48,transmission_toyota_camry_2007,...,0
7,comfort_toyota_camry_2007,...,0


In [19]:
cluster_centers=km_cluster.cluster_centers_
print('cluster_centers shape :', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 4611)
[[0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.01005322 0.         0.         ... 0.00706287 0.         0.        ]]


In [20]:

#군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함.
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
  cluster_details={}

  #cluster_centers array 값이 큰 순으로 정렬된 인덱스 값을 반환
  #군집 중심점 별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함
  centroid_feature_ordered_ind=cluster_model.cluster_centers_.argsort()[:, ::-1]

  #개별 군집별로 반복하며 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
  for cluster_num in range(clusters_num):
    #개별 군집별 정보를 담을 데이터 초기화
    cluster_details[cluster_num]={}
    cluster_details[cluster_num]['cluster']=cluster_num

    #cluster_centers_.argsort()로 구한 인덱스를 이용해 top n 피처 단어를 구함
    top_feature_indexes=centroid_feature_ordered_ind[cluster_num, :top_n_features]
    top_features=[feature_names[ind] for ind in top_feature_indexes]

    #top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함.
    top_feature_values=cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

    #cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
    cluster_details[cluster_num]['top_features']=top_features
    cluster_details[cluster_num]['top_features_value']=top_feature_values
    filenames=cluster_data[cluster_data['cluster_label']==cluster_num]['filename']
    filenames=filenames.values.tolist()

    cluster_details[cluster_num]['filenames']=filenames

  return cluster_details

In [21]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('##### Cluster {0}'.format(cluster_num))
    print('Top features:', cluster_detail['top_features'])
    print('reviews 파일명:', cluster_detail['filenames'][:7])
    print('==============================================')

In [22]:
feature_names=tfidf_vect.get_feature_names_out()

cluster_details=get_cluster_details(cluster_model=km_cluster, cluster_data=document_df, feature_names=feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)

##### Cluster 0
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
reviews 파일명: ['comfort_toyota_camry_2007', 'comfort_honda_accord_2008', 'mileage_honda_accord_2008', 'performance_honda_accord_2008', 'interior_toyota_camry_2007', 'interior_honda_accord_2008', 'gas_mileage_toyota_camry_2007']
##### Cluster 1
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
reviews 파일명: ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'location_holiday_inn_london', 'food_swissotel_chicago', 'parking_bestwestern_hotel_sfo', 'location_bestwestern_hotel_sfo', 'free_bestwestern_hotel_sfo']
##### Cluster 2
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
reviews 파일명: ['buttons_amazon_kindle', 'display_garmin_nuvi_255W_gps', 'accuracy_garmin_nuvi_255W_gps', 'battery-life_netbook_1005

10. 텍스트 분석 실습 - Mercari Price Suggestion Challenge

In [25]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [27]:
!p7zip -d './gdrive/My Drive/Colab Notebooks/ESAA_OB/train.tsv.7z'from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

mercari_df=pd.read_csv('train.tsv', sep='\t')
print(mercari_df.shape)
mercari_df.head(3)

/usr/bin/p7zip: cannot read ./gdrive/My Drive/Colab Notebooks/ESAA_OB/train.tsv.7zfrom


OSError: ignored

In [ ]:
import numpy as np

y_train_df=np.log1p(y_train_df)
sns.distplot(y_train_df, kde=False)

In [ ]:
#apply lambda 에서 호출되는 대, 중, 소 분할 함수 생성, 대, 중, 소 값을 리스트로 반환
def split_cat(category_name):
  try:
    return category_name.split('/')
  except:
    return ['Other_Null', 'Other_Null', 'Other_Null'] #대, 중, 소에 모두 other_null 넣기기

#위의 split_cat()을 apply lambda에서 호출해 대, 중, 소 칼럼을 mercari_df에 생성.
mercari_df['cat_dae'], mercari_df['cat_jung'], mercari_df['cat_so']=zip(*mercari_df['category_name'].apply(lambda x:split_

In [ ]:
#대분류만 값의 유형과 건수를 살펴보고, 중분류, 소분류는 값의 유형이 많으므로 분류 개수만 추출
print('대분류 유형 :\n', mercari_df['cat_dae'].value_counts())
print('중분류 개수 :', mercari_df['cat_jung'].unique())
print('소분류 개수 :', mercari_df['cat_so'].unique())

In [ ]:
mercari_df['brand_name']=mercari_df['brand_name'].fillna(value='Other_Null')
mercari_df['category_name']=mercari_df['category_name'].fillna(value='Other_Null')
mercari_df['item_description']=mercari_df['item_description'].fillna(value='Other_Null')
     

#각 칼럼별로 null값 건수 확인, 모두 0이 나옴
mercari_df.isnull().sum()

In [ ]:
#name 속성 피처 벡터화 변환
cnt_vec=CountVectorizer()
X_name=cnt_vec.fit_transform(mercari_df.name)

#item_description 피처 벡터화 변환
tfidf_descp=TfidfVectorizer(max_features=50000, ngram_range=(1,3), stop_words='english')
X_descp=tfidf_descp.fit_transform(mercari_df['item_description'])

print('name vectorization shape:', X_name.shape)
print('item_description vectorization shape:', X_descp.shape)

In [ ]:
from sklearn.preprocessing import LabelBinarizer

#brand_name, item_condition_id, shipping 각 피처들을 희소 행렬 원-핫 인코딩 변환
lb_brand_name=LabelBinarizer(sparse_output=True)
X_brand=lb_brand_name.fit_transform(mercari_df['brand_name'])
lb_item_cond_id=LabelBinarizer(sparse_output=True)
X_item_cond_id=lb_item_cond_id.fit_transform(mercari_df['item_condition_id'])
lb_shipping=LabelBinarizer(sparse_output=True)
X_shipping=lb_shipping.fit_transform(mercari_df['shipping'])

#cat_dae 등 각 피처들을 희소 행렬 원-핫 인코딩 변환
lb_cat_dae=LabelBinarizer(sparse_output=True)
X_cat_dae=lb_cat_dae.fit_transform(mercari_df['cat_dae'])
lb_cat_jung=LabelBinarizer(sparse_output=True)
X_cat_jung=lb_cat_jung.fit_transform(mercari_df['cat_jung'])
lb_cat_so=LabelBinarizer(sparse_output=True)
X_cat_so=lb_cat_so.fit_transform(mercari_df['cat_so'])

릿지 회귀 모델 구축 및 평가

In [ ]:
def rmsle(y, y_pred):
  #underflow, overflow를 막기 위해 log1p로 rmsle 계산
  return np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(y_pred),2)))

def evaluate_org_price(y_test, preds):
  #원본 데이터는 log1p 변환되었으므로 expm1으로 원복 필요.
  preds_exmpm=np.expm1(preds)
  y_test_exmpm=np.expm1(y_test)

  #rmsle로 RMSLE 값 추출
  rmsle_result=rmsle(y_test_exmpm, preds_exmpm)
  return rmsle_result

In [ ]:
import gc
from scipy.sparse import hstack

def model_train_predict(model, matrix_list):
  #scipy.sparse 모듈의 hstack을 이용해 희소 행렬 결합
  X=hstack(matrix_list).tocsr()

  X_train, X_test, y_train, y_test=train_test_split(X, mercari_df['price'], test_size=0.2, random_state=156)

  #모델 학습 및 예측
  model.fit(X_train, y_train)
  preds=model.predict(X_test)

  del X, X_train, X_test, y_train
  gc.collect()

  return preds, y_test
     

In [ ]:
linear_model=Ridge(solver='lsqr', fit_intercept=False)

sparse_matrix_list=(X_name, X_brand, X_item_cond_id,
                    X_shipping, X_cat_dae, X_cat_jung, X_cat_so)
linear_preds, y_test=model_train_predict(model=linear_model, matrix_list=sparse_matrix_list)
print('item description을 제외했을 때 rmsle 값 ;', evaluate_org_price(y_test, linear_preds))

sparse_matrix_list=(X_descp, X_name, X_brand, X_item_cond_id,
                    X_shipping, X_cat_dae, X_cat_jung, X_cat_so)
linear_preds, y_test=model_train_predict(model=linear_model, matrix_list=sparse_matrix_list)
print('item description을 포함함했을 때 rmsle 값 ;', evaluate_org_price(y_test, linear_preds))

LGBM 회귀 모델 구축과 앙상블을 이용한 최종 예측 평가

In [ ]:
from lightgbm import LGBMRegressor

sparse_matrix_list=(X_descp, X_name, X_brand, X_item_cond_id,
                    X_shipping, X_cat_dae, X_cat_jung, X_cat_so)

lgbm_model=LGBMRegressor(n_estimators=100, learning_rate=0.5, num_leaves=125, random_state=156)
lgbm_preds, y_test=model_train_predict(model=lgbm_model, matrix_list=sparse_matrix_list)
print('lgbm rmsle 값 ;', evaluate_org_price(y_test, lgbm_preds))

In [ ]:
preds=lgbm_preds*0.45+linear_preds*0.55
print('lgbm과 ridge를 앙상블 한 최종 rmsle 값 :', evaluate_org_price(y_test, preds))